# Making Top Song Prediction

***

## Importing Data with SQLite and cleaning table with Pandas

In [20]:
import sqlite3
import pandas as pd

In [79]:
# Connect to DB
conn = sqlite3.connect("spotify.db") 

# Grab tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# Select Table
table_name = tables.iloc[1, 0]  # second table with lowercase column names, 'spotify'

                           name
0                        tracks
1                       spotify
2  spotify_dropped_ids_pictures


In [122]:
# Read into pandas df
df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn) # Takes a sql query and sql connection

# Peek the tables
print("\nFirst Entries:")
# print(df.head())
print(df.head().to_string(index=False)) # remove line wrapping in notebook
print(len(df))


First Entries:
                  track_name         artist_name           album_name track_duration  track_popularity track_release_date  explicit         isrc                  artist_genre  artist_popularity  artist_follower_count                                                    album_picture                                                   artist_picture               album_id               track_id              artist_id  top_songs_rating  top_artist_ranking           date_added
                     Sarcasm          Get Scared    Best Kind Of Mess           3:18                67         2010-01-01         1 USUM71021465 emo, metalcore, post-hardcore                 58                 764241 https://i.scdn.co/image/ab67616d0000b27381c8341abbed9421975ca71d https://i.scdn.co/image/ab6761610000e5eb0aaa9d35794ed0470d2f5072 3xkOfH5nqIMm4b8qDWlzLj 40i81AcCCxiVEFUWElcjZC 5r9uIJCoquVtEkCkvcn8Ez                 1                21.0 2023-05-02T19:26:56Z
                 Glass Heart      

In [123]:
# Data Cleaning

# Drop IDs and Link Features
df.drop(columns=[
    'album_picture',
    'artist_picture',
    'album_id',
    'track_id',
    'artist_id',
    'isrc',
    'date_added' # Half of my dataset is missing this value, will have to ignore temporal context of tracks
], inplace=True)

# Extract Year and Month from Date Features (numerically)

df['track_release_date'] = pd.to_datetime(df['track_release_date'], errors='coerce')
# df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Overwrite with numeric YYYYMM (int)
df = df[df['track_release_date'].notna()] # Drop NULLS, rare (only 235 entries)

df['track_release_date'] = df['track_release_date'].dt.strftime('%Y%m').astype(int)

###

# top_artist_ranking type double -> int
df['top_artist_ranking'] = df['top_artist_ranking'].fillna(0).astype(int)
df['top_artist_ranking'] = df['top_artist_ranking'].astype(int)


# Convert 'track_duration' from MM:SS to total seconds (int)
df['track_duration'] = df['track_duration'].str.split(':').apply(
    lambda x: int(x[0]) * 60 + int(x[1])
)

print(df.head().to_string(index=False)) # remove line wrapping in notebook
print(len(df))

                  track_name         artist_name           album_name  track_duration  track_popularity  track_release_date  explicit                  artist_genre  artist_popularity  artist_follower_count  top_songs_rating  top_artist_ranking
                     Sarcasm          Get Scared    Best Kind Of Mess             198                67              201001         1 emo, metalcore, post-hardcore                 58                 764241                 1                  21
                 Glass Heart             Caskets           Lost Souls             224                68              202108         0      metalcore, post-hardcore                 61                 209521                 2                  14
                      weapon Against The Current                fever             197                59              202107         0                      pop punk                 64                 532952                 3                  12
One Hundred Sleepless Ni

## Defining target variable - my_top_songs (Binary Classification)

### Top Song Definition: **Top 10%** of my total saved songs (Quintile) (~600 songs)

In [126]:
# Convert the top_songs_rating feature into a binary feature, where positive when a song is in the top 10% of my songs

cutoff = int(len(df) * 0.10)  # top 10% of songs
df['my_top_songs'] = df['top_songs_rating'] <= cutoff
df['my_top_songs'] = df['my_top_songs'].astype(int)  # Convert boolean to 0/1

df.drop(columns=['top_songs_rating'], inplace=True) # cleanup

print(df.head().to_string(index=False)) # remove line wrapping in notebook

                  track_name         artist_name           album_name  track_duration  track_popularity  track_release_date  explicit                  artist_genre  artist_popularity  artist_follower_count  top_artist_ranking  my_top_songs
                     Sarcasm          Get Scared    Best Kind Of Mess             198                67              201001         1 emo, metalcore, post-hardcore                 58                 764241                  21             1
                 Glass Heart             Caskets           Lost Souls             224                68              202108         0      metalcore, post-hardcore                 61                 209521                  14             1
                      weapon Against The Current                fever             197                59              202107         0                      pop punk                 64                 532952                  12             1
One Hundred Sleepless Nights     Pierce 

In [127]:
df.to_csv("checking_in.csv", index=False)

In [ ]:
# Categorical Data Encoding

# Needing some help here, need to take an NLP class


# Label Encoding - [0,1,2,3,4,5]
# Target Encoding


# Most imporant features?
# - Genre: Need to specially handle this encoding
# - 